<a href="https://colab.research.google.com/github/ArjunJSP/ELF_combos/blob/eccentricity-combos/diaphragm_combos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import math
from google.colab import data_table
import itertools
from itertools import permutations
from itertools import accumulate


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to a specific folder in your Google Drive
%cd /content/drive/My Drive/mountlocation

Mounted at /content/drive
/content/drive/My Drive/mountlocation


#Input Files for Editing

In [4]:
#e2k File to write-to (currently not using)
e2k_file = 'RSFH_PODIUM_V11.2.4.e2k'

#ETABS Excel output of story forces for RSX-ELF, RSY-ELF, and 00_SEISMASS, as well as Mass Summary by Story Table
file_path = "StoryForces_PODIUM_V11.2.4.xlsx"

#Story Forces and Static Load Case scale factors

Data defined in this section include:
*   pv_storyshears_X
*   pv_storyshears_Y
*   pv_seismass
*   base_shear_coeff_VX
*   base_shear_coeff_VY






In [5]:
#Seismic Factors for Fpx calculation:
Sds = 1.17  #updated Sds
Ie = 1.5

In [6]:
#######evaluate units more carefully for the excel sheet - or make sure they are exported to access using the right units!
# Read the first two rows to get headers and sub-headers
headers = pd.read_excel(file_path, sheet_name="Story Forces", header=[1])

# Read the actual data, skipping the first two rows used for headers
df_storyforces = pd.read_excel(file_path, sheet_name="Story Forces", skiprows=[0, 1,2])

# Set the MultiIndex columns
df_storyforces.columns = headers.columns

In [7]:
#######evaluate units more carefully for the excel sheet - or make sure they are exported to access using the right units!
# Read the first two rows to get headers and sub-headers
headers = pd.read_excel(file_path, sheet_name="Mass Summary by Story", header=[1])

# Read the actual data, skipping the first two rows used for headers
df_masssummary = pd.read_excel(file_path, sheet_name="Mass Summary by Story", skiprows=[0, 2])

df_masssummary['P'] = df_masssummary['UX']*32.2/1000 #multiply by gravity and convert to kips (units must be lb-s^2/ft for mass)
df_masssummary['P_Total'] = df_masssummary['P'].cumsum(axis = 0, skipna = True) #cumulative mass added up along building
dropindex = df_masssummary.loc[df_masssummary['Story'] == 'L00'].index[0]
df_masssummary = df_masssummary.drop(dropindex, axis=0)

In [8]:
#required inputs:
num_levels = len(df_storyforces['Story'].drop_duplicates())

In [9]:
#collapse story force output into useable dataframes
relevant_cols = ['Story','Output Case','Step Type','Location','VX','VY']
pv_storyshears_X = df_storyforces[relevant_cols]
pv_storyshears_X = pv_storyshears_X[pv_storyshears_X['Location'].isin(['Bottom'])]
pv_storyshears_X = pv_storyshears_X[pv_storyshears_X['Step Type'].isin(['Max'])]
pv_storyshears_X = pv_storyshears_X[pv_storyshears_X['Output Case'].isin(['RSX-STR (ELF)'])]

pv_storyshears_Y = df_storyforces[relevant_cols]
pv_storyshears_Y = pv_storyshears_Y[pv_storyshears_Y['Location'].isin(['Bottom'])]
pv_storyshears_Y = pv_storyshears_Y[pv_storyshears_Y['Step Type'].isin(['Max'])]
pv_storyshears_Y = pv_storyshears_Y[pv_storyshears_Y['Output Case'].isin(['RSY-STR (ELF)'])]

relevant_cols = ['Story','Output Case','Location','P']
pv_seismass = df_storyforces[relevant_cols]
pv_seismass = pv_seismass[pv_seismass['Location'].isin(['Bottom'])]
pv_seismass = pv_seismass[pv_seismass['Output Case'].isin(['00_SEISMIC MASS'])]

In [10]:
# #Calculate the static base shear coefficients for X
# base_shear_coeff_VX = pv_storyshears_X['Story'].reset_index(drop=True).to_frame()
# x_coeff = ( pv_storyshears_X['VX'].reset_index(drop=True)-pv_storyshears_X['VX'].shift(1).fillna(0).reset_index(drop=True) ) / ( pv_seismass['P'].reset_index(drop=True)-pv_seismass['P'].shift(1).fillna(0).reset_index(drop=True) )
# x_coeff = x_coeff.rename("Coeff").to_frame()
# base_shear_coeff_VX = base_shear_coeff_VX.merge(x_coeff, left_index=True, right_index=True)

# #Calculate the static base shear coefficients for Y
# base_shear_coeff_VY = pv_storyshears_Y['Story'].reset_index(drop=True).to_frame()
# y_coeff = ( pv_storyshears_Y['VY'].reset_index(drop=True)-pv_storyshears_Y['VY'].shift(1).fillna(0).reset_index(drop=True) ) / ( pv_seismass['P'].reset_index(drop=True)-pv_seismass['P'].shift(1).fillna(0).reset_index(drop=True) )
# y_coeff = y_coeff.rename("Coeff").to_frame()
# base_shear_coeff_VY = base_shear_coeff_VY.merge(y_coeff, left_index=True, right_index=True)


In [11]:
#Calculate the static base shear coefficients for X
base_shear_coeff_VX = pv_storyshears_X['Story'].reset_index(drop=True).to_frame()
x_coeff = ( pv_storyshears_X['VX'].reset_index(drop=True)-pv_storyshears_X['VX'].shift(1).fillna(0).reset_index(drop=True) ) / (df_masssummary['P'])
x_coeff = x_coeff.rename("Coeff").to_frame()
base_shear_coeff_VX = base_shear_coeff_VX.merge(x_coeff, left_index=True, right_index=True)

#Calculate the static base shear coefficients for Y
base_shear_coeff_VY = pv_storyshears_Y['Story'].reset_index(drop=True).to_frame()
y_coeff = ( pv_storyshears_Y['VY'].reset_index(drop=True)-pv_storyshears_Y['VY'].shift(1).fillna(0).reset_index(drop=True) ) /  (df_masssummary['P'])
y_coeff = y_coeff.rename("Coeff").to_frame()
base_shear_coeff_VY = base_shear_coeff_VY.merge(y_coeff, left_index=True, right_index=True)

#Permutation Parameters
Generate all permutations for load combination based on the following parameters:
- direction (X,Y)
- level
- eccentricity (P,N)


In [12]:
#function create parameter array for level names
def create_l_array(n):
    """
    Generate an array of text elements with the letter 'L' concatenated with numbers from 01 to n.
    Adds a leading zero for numbers less than 10.

    :param n: Length of the array.
    :return: List of strings in the format 'L01', 'L02', ..., 'Ln'.
    """
    return [f"L{str(i).zfill(2)}" for i in range(n, 0, -1)]


In [13]:
#axis parameters
param_axis = ['X','Y']

#level parameters
param_level = create_l_array(num_levels)

#direction parameters
param_direction = ['P','N']

#eccentricity parameters
param_ecc = ['P','N']
ecc_percent = 0.05

In [14]:
#function to create a list of permutations
def generate_permutations(*criteria):
    """
    Generate all possible permutations from multiple criteria lists.

    :param criteria: Variable number of lists containing elements for permutation.
    :return: List of tuples representing all possible permutations.
    """
    return list(itertools.product(*criteria))

In [15]:
#permutations to compute:
# 0: axis (X,Y)
# 1: direction (positive, negative)
# 2: eccentricity (positive, negative)
# 3: level
permutations = generate_permutations(param_axis, param_direction, param_ecc, param_level)
n = len(permutations)

# Calculate Fpx Factors


*   Fpx_SF = table of scale factors for each floor, for each direction



In [16]:
#Fpx calcualation based on ASCE 7-16 12.10.1.1: Diaphragm Design Forces (RE-WRITE USING MASS SOURCE)

Wpx = df_masssummary['P']
Fx_X = (pv_storyshears_X.VX - pv_storyshears_X.VX.shift(1).fillna(0.0)).reset_index(drop=True)
Fx_Y = (pv_storyshears_Y.VY - pv_storyshears_Y.VY.shift(1).fillna(0.0)).reset_index(drop=True)

Wi = df_masssummary['P_Total']
Fi_x = pv_storyshears_X.VX.reset_index(drop=True)
Fi_y = pv_storyshears_Y.VY.reset_index(drop=True)

Fpx_X = [None] * num_levels
Fpx_Y = [None] * num_levels

for i in range(num_levels):
  Fpx_X[i] = Fi_x[i]/Wi[i]*Wpx[i]       # 12.10-1
  if Fpx_X[i] < 0.2*Sds*Ie*Wpx[i]:
    Fpx_X[i] = 0.2*Sds*Ie*Wpx[i]        # 12.10-2
  if Fpx_X[i] > 0.4*Sds*Ie*Wpx[i]:
    Fpx_X[i] = 0.4*Sds*Ie*Wpx[i]        # 12.10-3

for i in range(num_levels):
  Fpx_Y[i] = Fi_y[i]/Wi[i]*Wpx[i]       # 12.10-1
  if Fpx_Y[i] < 0.2*Sds*Ie*Wpx[i]:
    Fpx_Y[i] = 0.2*Sds*Ie*Wpx[i]        # 12.10-2
  if Fpx_Y[i] > 0.4*Sds*Ie*Wpx[i]:
    Fpx_Y[i] = 0.4*Sds*Ie*Wpx[i]        # 12.10-3

#Story Scale Factor coefficient
Fpx_X_SF = (Fpx_X / Fx_X).round(2)
Fpx_Y_SF = (Fpx_Y / Fx_Y).round(2)

Fpx_SF = pd.DataFrame([Fpx_X_SF,Fpx_Y_SF]) # .rename(columns=param_level)
Fpx_SF.columns = param_level
Fpx_SF.index = ['X','Y']

#Generate Load Patterns

*   pattern_array = names for all of the required load patterns, based on an individual pattern for each load permutation
*   load_pattern_array = load_pattern_array1 + load_pattern_array2, where both are the component lines for the total line section that is the load pattern array



In [17]:
#first line in load pattern definitions

pattern_array = [None] * n
load_pattern_array1 = [None] * n
for i in range(n):
  pattern_array[i] = ' "ELF-EQ'+ permutations[i][0] + permutations[i][1] + "-" + permutations[i][2] + "-" + permutations[i][3] + '"'
  load_pattern_array1[i] = "  LOADPATTERN" + pattern_array[i] + '  TYPE  "Seismic"  SELFWEIGHT  0'

In [18]:
#second line in load pattern definitions

## create a list of length 'n' to define positive (+) or negative (-) eccentricity
ecc_array = [None] * n
for i in range(n):
  if permutations[i][2] == "P":
    ecc_array[i] = 'DIR "' + permutations[i][0] + '+ECC"  ECC ' + str(ecc_percent)
  else:
    ecc_array[i] = 'DIR "' + permutations[i][0] + '-ECC"  ECC ' + str(ecc_percent)

## create a list of length 'n' to define TOPSTORY and BOTTOMSTORY

df_stories = pd.concat([df_storyforces['Story'].drop_duplicates().reset_index(drop=True).to_frame().rename(columns={"Story": "TOPSTORY"}), df_storyforces['Story'].drop_duplicates().shift(-1).fillna("L00").reset_index(drop=True).to_frame().rename(columns={"Story": "BOTTOMSTORY"})], axis=1)
# df_stories =  pd.concat([df_storyforces['Story'].drop_duplicates().reset_index(drop=True).to_frame().rename(columns={"Story": "TOPSTORY"}), df_storyforces['Story'].drop_duplicates().reset_index(drop=True).to_frame().rename(columns={"Story": "BOTTOMSTORY"})], axis=1)

story_array = [None] * n
for i in range(n):
  TOPSTORY = permutations[i][3]
  BOTTOMSTORY = df_stories.loc[df_stories['TOPSTORY'] == TOPSTORY, 'BOTTOMSTORY'].iloc[0]
  story_array[i] = 'TOPSTORY' + ' "' + TOPSTORY + '"    ' + 'BOTTOMSTORY' + ' "' + BOTTOMSTORY + '"    '

## assign SHEARCOEFF based on TOPSTORY definition
shear_array = [None] * n
shear_values = [None] * n
for i in range(n):
  if permutations[i][0] == "X":
    shear_values[i] = base_shear_coeff_VX.loc[base_shear_coeff_VX['Story'] == permutations[i][3], 'Coeff'].iloc[0].round(4)
  else:
    shear_values[i] = base_shear_coeff_VY.loc[base_shear_coeff_VY['Story'] == permutations[i][3], 'Coeff'].iloc[0].round(4)

# FYI: shear coefficients cannot be negative values in User Coefficient Load Patterns
  shear_array[i] = 'SHEARCOEFF ' + str(shear_values[i]) + '  HEIGHTEXPONENT 1'

# create final array
load_pattern_array2 = [None] * n

for i in range(n):
  load_pattern_array2[i] = '  SEISMIC ' + pattern_array[i] + '  "User Coefficient"    ' + ecc_array[i] + '  ' + story_array[i] + shear_array[i]


In [19]:
load_pattern_array = load_pattern_array1 +  load_pattern_array2

#Generate Load Cases

In [20]:
loadcase_line1_array = [None] * n
loadcase_line2_array = [None] * n
load_case_array = [None] * n*2
for i in range(n):
  loadcase_line1_array[i] = '  LOADCASE ' + pattern_array[i] + '  TYPE  "Linear Static"  INITCOND  "PRESET"  '
  if permutations[i][1] == "P":
    loadcase_line2_array[i] = '  LOADCASE ' + pattern_array[i] + '  LOADPAT  ' + pattern_array[i] + '  SF  1'
  else:
    loadcase_line2_array[i] = '  LOADCASE ' + pattern_array[i] + '  LOADPAT  ' + pattern_array[i] + '  SF  -1'


for i in range(n*2):
  load_case_array[::2] = loadcase_line1_array
  load_case_array[1::2] = loadcase_line2_array

# Generate Utility Load Combinations

In [103]:
utility_combo1 = [];

type = '"Linear Add"'

#permutations required for utility combo 1 capturing positive/negative direction and eccentricity for a given axis (X,Y) and floor level
utility_combo1_permutations = generate_permutations(param_axis,param_direction,param_ecc,param_level) #additional param_level to add individual levels for each Fpx load combo
fpx_combo_permutations = generate_permutations(param_level)

utility_combo1_len = len(utility_combo1_permutations)
fpx_combo_len = len(fpx_combo_permutations)

#utility load combo1 array defining all the combos we need for utility combo1
utility_combo1_array = [None] * utility_combo1_len

#define utility combo
for i in range(utility_combo1_len):
  utility_combo1_array[i] = '"ELF-EQ' + utility_combo1_permutations[i][0] + '-' + utility_combo1_permutations[i][1] + '-' + utility_combo1_permutations[i][2]  + '-Fpx-' + utility_combo1_permutations[i][3] +'"'

#define combos/cases included in utility combo
for i in range(utility_combo1_len):
    utility_combo1.append('  COMBO  ' + utility_combo1_array[i] + '  TYPE ' + type)
    #Fpx scale factor depending on axis of force (X,Y)
    SF = Fpx_SF.iloc[Fpx_SF.index.get_loc(utility_combo1_permutations[i][0]), Fpx_SF.columns.get_loc(utility_combo1_permutations[i][3])]

    #apply Fpx only if the floor matches the loop increment
    for j in range(fpx_combo_len):
      if ((fpx_combo_permutations[j][0] == utility_combo1_permutations[i][3])):
        utility_combo1.append( '  COMBO  ' + utility_combo1_array[i] + '  LOADCASE  ' + pattern_array[i] + '  SF ' + str(SF) )
      else:
        utility_combo1.append( '  COMBO  ' + utility_combo1_array[i] + '  LOADCASE  ' + pattern_array[i] + '  SF 1 ' )


In [21]:
# #Envelope all eccentricity and direction on a given floor, for both X and Y

# utility_combo1 = [];

# type = '"Envelope"'

# #permutations required for utility combo 1 capturing positive/negative direction and eccentricity for a given axis (X,Y) and floor level
# utility_combo1_permutations = generate_permutations(param_axis, param_level)
# utility_combo1_len = len(utility_combo1_permutations)

# #utility load combo1 array defining all the combos we need for utility combo1
# utility_combo1_array = [None] * utility_combo1_len

# for i in range(utility_combo1_len):
#   utility_combo1_array[i] = '"ELF-EQ' + utility_combo1_permutations[i][0] + '-' + utility_combo1_permutations[i][1]+'"'

# for x in range(utility_combo1_len):
#     utility_combo1.append('  COMBO  ' + utility_combo1_array[x] + '  TYPE ' + type)
#     for i in range(n):
#       if ((permutations[i][0] == utility_combo1_permutations[x][0]) and (permutations[i][3] == utility_combo1_permutations[x][1])):
#          utility_combo1.append( '  COMBO  ' + utility_combo1_array[x] + '  LOADCASE  ' + pattern_array[i] + '  SF 1 ')

In [22]:
# #Linear-Add all Levels while adding Fpx scale factor to single floors

# utility_combo2 = [];

# type = '"Linear Add"'

# #permutations required for utility combo 2 capturing Fpx scaling for single floor for each direction (X,Y) with linear add of all floors
# utility_combo2_permutations = generate_permutations(param_axis, param_level)
# utility_combo2_len = len(utility_combo2_permutations)

# #utility load combo2 array defining all the combos we need for utility combo1
# utility_combo2_array = [None] * utility_combo2_len

# for i in range(utility_combo2_len):
#   utility_combo2_array[i] = '"ELF-EQ' + utility_combo2_permutations[i][0] + '-Fpx-' + utility_combo2_permutations[i][1]+'"'

# #create utility laod combo2 array
# for x in range(utility_combo2_len):
#     utility_combo2.append('  COMBO  ' + utility_combo2_array[x] + '  TYPE ' + type)

#     #Fpx scale factor depending on axis of force (X,Y)
#     SF = Fpx_SF.iloc[Fpx_SF.index.get_loc(utility_combo2_permutations[x][0]), Fpx_SF.columns.get_loc(utility_combo2_permutations[x][1])]

#     for i in range(utility_combo2_len):

#       #apply Fpx only if the floor matches the loop increment
#       if ((utility_combo1_permutations[i][0] == utility_combo2_permutations[x][0]) and (utility_combo1_permutations[i][1] == utility_combo2_permutations[x][1])):
#          utility_combo2.append( '  COMBO  ' + utility_combo2_array[x] + '  LOADCOMBO  ' + utility_combo1_array[i] + '  SF ' + str(SF) )

#       elif (utility_combo1_permutations[i][0] == utility_combo2_permutations[x][0]):
#          utility_combo2.append( '  COMBO  ' + utility_combo2_array[x] + '  LOADCOMBO  ' + utility_combo1_array[i] + '  SF 1 ' )


In [102]:
#Envelope by direction (X,Y) for all floors in order to get final

utility_combo3 = [];

type = '"Envelope"'

#permutations required for utility combo 2 capturing Fpx scaling for single floor for each direction (X,Y) with linear add of all floors
utility_combo3_permutations = generate_permutations(param_axis)
utility_combo3_len = len(utility_combo3_permutations)

#utility load combo3 array defining all the combos we need for utility combo1
utility_combo3_array = [None] * utility_combo3_len

for i in range(utility_combo3_len):
  utility_combo3_array[i] = '"ELF-EQ' + utility_combo3_permutations[i][0] +'"'

#create utility laod combo2 array
for x in range(utility_combo3_len):
  utility_combo3.append('  COMBO  ' + utility_combo3_array[x] + '  TYPE ' + type)

  for i in range(utility_combo2_len):

    #apply Fpx only if the floor matches the loop increment
    if ((utility_combo2_permutations[i][0] == utility_combo3_permutations[x][0])):
       utility_combo3.append( '  COMBO  ' + utility_combo3_array[x] + '  LOADCOMBO  ' + utility_combo2_array[i] +'  SF 1 ' )

TypeError: can only concatenate str (not "NoneType") to str

In [24]:
load_combo_array = utility_combo1 + utility_combo2 + utility_combo3

# Write to a .txt file (temporary)

#Insert Load Patterns, Cases, and Combos into .e2k file

In [25]:
def find_and_replace(input_file, output_file, search_text, replacement_lines):
    """
    Replaces the first occurrence of lines starting with `search_text` with `replacement_lines`.

    :param input_file: Path to the input text file.
    :param output_file: Path to the output text file.
    :param search_text: The text to search for at the beginning of a line.
    :param replacement_lines: List of replacement lines (without '\n').
    """
    # Ensure replacement lines end with '\n'
    replacement_lines = [line + "\n" for line in replacement_lines]

    # Read the file
    with open(input_file, "r") as file:
        lines = file.readlines()

    # Find the first occurrence index of `search_text`
    first_index = next((i for i, line in enumerate(lines) if line.startswith(search_text)), -1)

    # Remove all occurrences of `search_text`
    filtered_lines = [line for line in lines if not line.startswith(search_text)]

    # Insert replacement lines at the position of the first removed occurrence
    if first_index != -1:
        filtered_lines[first_index:first_index] = replacement_lines  # Insert at first_index

    # Write the modified content to a new file (or overwrite the original)
    with open(output_file, "w") as file:
        file.writelines(filtered_lines)

    print(f"File processed successfully. Check {output_file}")


In [26]:
# Define file paths
input_file = e2k_file  # Replace with your actual file name
output_file = "ELF-"+e2k_file  # Output file name

# Replace load patterns pt. 1
find_and_replace(input_file, "temp1"+output_file, '  LOADPATTERN "ELF-', load_pattern_array1)

# Replace load patterns pt. 2
find_and_replace("temp1"+output_file, "temp2"+output_file, '  SEISMIC "ELF-', load_pattern_array2)

# Replace load cases
find_and_replace("temp2"+output_file, "temp3"+output_file, '  LOADCASE "ELF-', load_case_array)

# Replace load combos
find_and_replace("temp3"+output_file, output_file, '  COMBO "ELF-', load_combo_array)


File processed successfully. Check temp1ELF-RSFH_PODIUM_V11.2.4.e2k
File processed successfully. Check temp2ELF-RSFH_PODIUM_V11.2.4.e2k
File processed successfully. Check temp3ELF-RSFH_PODIUM_V11.2.4.e2k
File processed successfully. Check ELF-RSFH_PODIUM_V11.2.4.e2k


In [27]:
# with open("testloads2.txt", "w") as file:
#     for item in load_pattern_array:
#         file.write(item + '\n')  # Write each item on a new line

# with open("testloads2.txt", "a") as file:
#     for item in load_case_array:
#         file.write(item + '\n')  # Write each item on a new line

# with open("testloads2.txt", "a") as file:
#     for item in load_combo_array:
#         file.write(item + '\n')  # Write each item on a new line
